In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

# = = = = = = = = = = = = = = =

option = ChromeOptions()

# = = = = = = = = = = = = = = =

option.add_extension(Proxy.get_Proxy_Selenium('coroutine'))
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument(UA.get_User_Agent_Selenium())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)

                browser.maximize_window()

                browser.get(url)
                
                # = = = = = = = = = = = = = = =
                
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)
                browser.execute_script('window.scrollTo(0, -document.body.scrollHeight);')

                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//h1[@class="part-detail-title"]')))
                title = browser.find_element(by=By.XPATH, value='//h1[@class="part-detail-title"]').text.strip()

                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//p[@class="part-detail-title-number"]')))
                part_number = browser.find_element(by=By.XPATH, value='//p[@class="part-detail-title-number"]').text.replace('Part Number:', '').strip()

                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//p[@class="item-description"]')))
                description = browser.find_element(by=By.XPATH, value='//p[@class="item-description"]').get_attribute('outerHTML')

                html = etree.HTML(description)
                description = ' '.join(html.xpath('//p[@class="item-description"]/text()')).strip()
                
                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//img[@id="primaryImage"]')))
                src = browser.find_element(by=By.XPATH, value='//img[@id="primaryImage"]').get_attribute('src').strip()
                
                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="attribute-container flex-container flex-wrap"]/div[@class="small-12 medium-9 large-6"]')))
                list_name = browser.find_elements(by=By.XPATH, value='//div[@class="attribute-container flex-container flex-wrap"]/div[@class="small-12 medium-9 large-6"]')
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="attribute-container flex-container flex-wrap"]/div[@class="small-12 medium-15 large-18"]')))
                list_value = browser.find_elements(by=By.XPATH, value='//div[@class="attribute-container flex-container flex-wrap"]/div[@class="small-12 medium-15 large-18"]')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': part_number,
                                         'Title': title,
                                         'Description': description,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src}])

                for name, value in zip(list_name, list_value):
                    df_temp.loc[0, name.text.strip()[:-1]] = value.text.strip()
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])
            
            browser.quit()

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(8):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：234

https://www.summitracing.com/parts/trd-0184  <->  [ok] - 剩余数量：226 - 当前时间：16:02:18
https://www.summitracing.com/parts/trd-42252  <->  [ok] - 剩余数量：225 - 当前时间：16:02:21
https://www.summitracing.com/parts/trd-46002  <->  [ok] - 剩余数量：224 - 当前时间：16:02:31
https://www.summitracing.com/parts/trd-48061  <->  [ok] - 剩余数量：223 - 当前时间：16:02:32
https://www.summitracing.com/parts/trd-46012  <->  [ok] - 剩余数量：222 - 当前时间：16:02:34
https://www.summitracing.com/parts/tsn-81071  <->  [ok] - 剩余数量：221 - 当前时间：16:02:40
https://www.summitracing.com/parts/trd-42235  <->  [ok] - 剩余数量：220 - 当前时间：16:02:41
https://www.summitracing.com/parts/tsn-81006  <->  [ok] - 剩余数量：219 - 当前时间：16:02:46
https://www.summitracing.com/parts/sum-csumcpgmls02  <->  [ok] - 剩余数量：218 - 当前时间：16:02:51
https://www.summitracing.com/parts/tsn-81073bk  <->  [ok] - 剩余数量：217 - 当前时间：16:02:52
https://www.summitracing.com/parts/tsn-81019  <->  [ok] - 剩余数量：216 - 当前时间：16:02:58
https://www.summitracing.com/parts/tsn-81072  <->  [ok] - 剩余数量：215 - 当

https://www.summitracing.com/parts/mez-ts419  <->  [ok] - 剩余数量：129 - 当前时间：16:09:41
https://www.summitracing.com/parts/mez-tss062  <->  [ok] - 剩余数量：128 - 当前时间：16:09:48
https://www.summitracing.com/parts/mil-21575  <->  [ok] - 剩余数量：127 - 当前时间：16:09:51
https://www.summitracing.com/parts/mez-ts119  <->  [ok] - 剩余数量：126 - 当前时间：16:09:51
https://www.summitracing.com/parts/mah-os32244  <->  [ok] - 剩余数量：125 - 当前时间：16:09:56
https://www.summitracing.com/parts/mil-21576  <->  [ok] - 剩余数量：124 - 当前时间：16:09:59
https://www.summitracing.com/parts/mil-21580  <->  [ok] - 剩余数量：123 - 当前时间：16:10:01
https://www.summitracing.com/parts/mil-30914  <->  [ok] - 剩余数量：122 - 当前时间：16:10:04
https://www.summitracing.com/parts/mil-30916  <->  [ok] - 剩余数量：121 - 当前时间：16:10:07
https://www.summitracing.com/parts/mil-31507  <->  [ok] - 剩余数量：120 - 当前时间：16:10:29
https://www.summitracing.com/parts/mil-30915  <->  [ok] - 剩余数量：119 - 当前时间：16:10:32
https://www.summitracing.com/parts/mil-41012  <->  [ok] - 剩余数量：118 - 当前时间：16:10:44
h

https://www.summitracing.com/parts/drt-31937142  <->  [ok] - 剩余数量：31 - 当前时间：16:21:08
https://www.summitracing.com/parts/cmb-08-0197  <->  [ok] - 剩余数量：30 - 当前时间：16:21:26
https://www.summitracing.com/parts/drt-31937221  <->  [ok] - 剩余数量：29 - 当前时间：16:21:36
https://www.summitracing.com/parts/drt-31937212  <->  [ok] - 剩余数量：28 - 当前时间：16:21:57
https://www.summitracing.com/parts/drt-31937242  <->  [ok] - 剩余数量：27 - 当前时间：16:22:06
https://www.summitracing.com/parts/drt-31947111  <->  [ok] - 剩余数量：26 - 当前时间：16:22:41
https://www.summitracing.com/parts/drt-31947112  <->  [ok] - 剩余数量：25 - 当前时间：16:22:53
https://www.summitracing.com/parts/drt-31947121  <->  [ok] - 剩余数量：24 - 当前时间：16:23:04
https://www.summitracing.com/parts/ctr-13-274blk  <->  [ok] - 剩余数量：23 - 当前时间：16:23:24
https://www.summitracing.com/parts/ctr-15-274  <->  [ok] - 剩余数量：22 - 当前时间：16:23:26
https://www.summitracing.com/parts/drt-31947122  <->  [ok] - 剩余数量：21 - 当前时间：16:23:32
https://www.summitracing.com/parts/drt-31947212  <->  [ok] - 剩余数量：2